# MLflow v3.0 Data Setup for Migration Testing

In [ ]:
# Install dependencies

!pip3 install mlflow==3.0
!pip3 install sagemaker-mlflow==0.2.0

In [ ]:
# import modules

import mlflow
from mlflow.models import infer_signature
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor

import numpy as np
from sklearn.linear_model import LinearRegression
import mlflow
from mlflow import MlflowClient

In [ ]:
# Configuration - Replace with your actual values
tracking_server_name = '<your-tracking-server-name>'
tracking_server_arn = "<your-tracking-server-arn>"
registered_model_name = "<your-model-name>"
mlflow.set_tracking_uri(tracking_server_arn)

In [ ]:
@mlflow.trace
def train_and_log_linear_regression_model(experiment_name):

    mlflow.set_experiment(experiment_name)
    # enable autologging
    mlflow.sklearn.autolog(registered_model_name=registered_model_name)
    
    with mlflow.start_span(name="data_preparation") as span:
        # prepare training data
        X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
        y = np.dot(X, np.array([1, 2])) + 3
        span.set_inputs({"X_shape": X.shape, "y_shape": y.shape})
        span.set_outputs({"data_ready": True})
    
    with mlflow.start_span(name="model_training") as span:
        model = LinearRegression()
        with mlflow.start_run() as run:
            # Log additional data
            mlflow.log_param("data_shape", X.shape)
            mlflow.log_param("model_type", "LinearRegression")
            mlflow.log_metric("training_samples", len(X))
            
            model.fit(X, y)
            span.set_inputs({"model": "LinearRegression", "training_data": X.shape})
            span.set_outputs({"model_trained": True})
            
            with mlflow.start_span(name="model_evaluation") as eval_span:
                # Log predictions and score
                predictions = model.predict(X)
                score = model.score(X, y)
                mlflow.log_metric("r2_score", score)
                mlflow.log_metric("mean_prediction", np.mean(predictions))
                
                # Log model coefficients
                mlflow.log_param("coefficients", model.coef_.tolist())
                mlflow.log_param("intercept", model.intercept_)
                
                eval_span.set_inputs({"test_data": X.shape})
                eval_span.set_outputs({"r2_score": score, "predictions": predictions.tolist()})

In [ ]:
def create_sample_experiments():
    exps = [f"linear-regression-baseline-{tracking_server_name}", f"model-performance-analysis-{tracking_server_name}", f"sklearn-migration-test-{tracking_server_name}", 
            f"data-validation-experiment-{tracking_server_name}", f"regression-model-comparison-{tracking_server_name}"]
    
    for exp in exps:
        train_and_log_linear_regression_model(exp)

In [ ]:
## Create Sample Experiments for Migration Testing

create_sample_experiments()

## MLflow v3.0 Prompt Creation for Migration Testing

In [ ]:
def create_prompt(prompt_name, template, tags=None, commit_message=None):
    """Create a prompt with version compatibility.
    
    Works with MLflow 2.21+ and 3.0+.
    Note: MLflow 2.16-2.20 do NOT support prompts.
    """
    import mlflow
    
    # Try MLflow 3.0+ API first
    try:
        import mlflow.genai
        prompt = mlflow.genai.register_prompt(
            name=prompt_name,
            template=template,
            tags=tags or {},
            commit_message=commit_message or f"Created {prompt_name}"
        )
        print(f"Created: {prompt_name} v{prompt.version}")
        return prompt
    except (ImportError, AttributeError):
        # Fallback for MLflow 2.21-2.x clients
        try:
            import warnings
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=FutureWarning)
                prompt = mlflow.register_prompt(
                    name=prompt_name,
                    template=template,
                    tags=tags or {},
                    commit_message=commit_message or f"Created {prompt_name}"
                )
            print(f"Created: {prompt_name} v{prompt.version}")
            return prompt
        except Exception as e:
            print(f"Failed: {prompt_name} - {e}")
            return None
    except Exception as e:
        print(f"Failed: {prompt_name} - {e}")
        return None

In [ ]:
def create_test_prompts():
    prompts_to_create = [
        {
            "name": f"summarization-prompt-{tracking_server_name}",
            "template": "Summarize the following text in {{max_words}} words or less:\n\n{{text}}",
            "tags": {"task": "summarization", "version": "v1"},
            "commit_message": "Initial summarization prompt"
        },
        {
            "name": f"qa-prompt-{tracking_server_name}",
            "template": "Answer the following question based on the context:\n\nContext: {{context}}\n\nQuestion: {{question}}\n\nAnswer:",
            "tags": {"task": "qa", "version": "v1"},
            "commit_message": "Initial Q&A prompt"
        },
        {
            "name": f"translation-prompt-{tracking_server_name}",
            "template": "Translate the following text from {{source_lang}} to {{target_lang}}:\n\n{{text}}",
            "tags": {"task": "translation", "version": "v1"},
            "commit_message": "Initial translation prompt"
        },
        {
            "name": f"classification-prompt-{tracking_server_name}",
            "template": "Classify the following text into one of these categories: {{categories}}\n\nText: {{text}}\n\nCategory:",
            "tags": {"task": "classification", "version": "v1"},
            "commit_message": "Initial classification prompt"
        },
        {
            "name": f"code-generation-prompt-{tracking_server_name}",
            "template": "Generate {{language}} code for the following task:\n\n{{task_description}}\n\nCode:",
            "tags": {"task": "code-generation", "version": "v1"},
            "commit_message": "Initial code generation prompt"
        }
    ]
    
    created_prompts = []
    for prompt_config in prompts_to_create:
        prompt = create_prompt(
            prompt_name=prompt_config["name"],
            template=prompt_config["template"],
            tags=prompt_config["tags"],
            commit_message=prompt_config["commit_message"]
        )
        if prompt:
            created_prompts.append(prompt)
    
    print(f"\nCreated {len(created_prompts)} prompts")
    return created_prompts

In [ ]:
def create_prompt_versions():
    prompt_name = f"multi-version-prompt-{tracking_server_name}"
    versions = [
        {"template": "Version 1: Simple prompt with {{variable}}", "tags": {"version": "v1", "status": "deprecated"}, "commit_message": "Initial version"},
        {"template": "Version 2: Improved prompt with {{variable}} and {{context}}", "tags": {"version": "v2", "status": "testing"}, "commit_message": "Added context parameter"},
        {"template": "Version 3: Production-ready prompt with {{variable}}, {{context}}, and {{instructions}}", "tags": {"version": "v3", "status": "production"}, "commit_message": "Production release"}
    ]
    
    created_versions = []
    for version_config in versions:
        prompt = create_prompt(prompt_name, version_config["template"], version_config["tags"], version_config["commit_message"])
        if prompt:
            created_versions.append(prompt)
    
    print(f"\nCreated {len(created_versions)} versions of {prompt_name}")
    return created_versions

In [ ]:
## Create sample prompts for migration testing
prompts = create_test_prompts()
versions = create_prompt_versions()